In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
from sklearn.metrics import f1_score

In [2]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)

In [3]:
def read_dataset(filename, splitter="\t"):
    data = []
    sentence = []
    tags = []
    with open(filename) as f:
        for line in f:
            if not line.isspace():
                word, tag = line.split(splitter)
                sentence.append(word)
                tags.append(tag.strip())
            else:
                data.append((sentence, tags))
                sentence = []
                tags = []
    return data

In [4]:
training_data = read_dataset("task1/train.tsv")

In [5]:
import pandas as pd
ner_data = pd.DataFrame(training_data, columns=['tokens', 'tags'])

In [6]:
ner_data

,tokens,tags
0,"[also, ,, i, have, recently, discovered, advil...","[O, O, O, O, O, O, B-Object, O, O, O, B-Predic..."
1,"[i, have, always, heard, that, motrin, is, bet...","[O, O, O, O, O, B-Object, O, B-Predicate, O, B..."
2,"[when, i, was, a, figure, skater, i, injuried,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,"[in, a, way, ,, halloween, is, even, better, t...","[O, O, O, O, B-Object, O, O, B-Predicate, O, B..."
4,"[i, think, halloween, is, actually, safer, tha...","[O, O, B-Object, O, O, B-Predicate, O, O, O, B..."
...,...,...
2329,"[isn, ', t, plastic, safer, than, wood, .]","[O, O, O, B-Object, B-Predicate, O, B-Object, O]"
2330,"[and, plastic, pallets, are, safer, than, wood...","[O, B-Object, O, O, B-Predicate, O, B-Object, ..."
2331,"[plastic, laminate, flooring, is, one, of, the...","[B-Object, O, B-Aspect, O, O, O, O, O, O, O, O..."
2332,"[plastic, has, long, been, considered, superio...","[B-Object, O, O, O, O, B-Predicate, O, B-Objec..."


In [7]:
label_list = []
for item in ner_data['tags']:
    label_list.extend(item)
label_list = list(set(label_list))
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Object',
 'B-Aspect',
 'I-Object',
 'I-Predicate',
 'B-Predicate',
 'I-Aspect']

In [8]:
from sklearn.model_selection import train_test_split
# ner_data = [extract_labels(item) for item in drugs]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

In [9]:
from datasets import load_dataset, load_metric
from datasets import Dataset, DatasetDict

In [10]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

DatasetDict({
    train: Dataset({
        features: ['tokens', 'tags', '__index_level_0__'],
        num_rows: 1867
    })
    test: Dataset({
        features: ['tokens', 'tags', '__index_level_0__'],
        num_rows: 467
    })
})

In [11]:
from transformers import AutoTokenizer
model_checkpoint = "cointegrated/rubert-tiny2"
#model_checkpoint = "sberbank-ai/ruRoberta-large"
batch_size = 16
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [12]:
tokenizer("Hello, this is one sentence!")

{'input_ids': [2, 9944, 16, 881, 550, 835, 15503, 5, 3], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [13]:
example = ner_data['train'][6]
print(example["tokens"])

['when', 'sony', 'announced', 'at', 'e3', 'how', 'much', 'better', 'it', 'was', 'than', 'microsoft', ',', 'it', 'reaffirmed', 'that', 'the', 'ps4', 'would', 'be', 'region', 'free', '.']


In [14]:
# from transformers import RobertaTokenizerFast
# tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base", add_prefix_space=True)
# tokenizer("Hello world")['input_ids']
# tokenizer(" Hello world")['input_ids']

In [15]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', 'when', 'son', '##y', 'announced', 'at', 'e', '##3', 'how', 'much', 'better', 'it', 'was', 'than', 'micro', '##so', '##ft', ',', 'it', 're', '##aff', '##ir', '##med', 'that', 'the', 'ps', '##4', 'would', 'be', 'region', 'free', '.', '[SEP]']


In [16]:
def tokenize_and_align_labels(examples, label_all_tokens=False):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [17]:
tokenize_and_align_labels(ner_data['train'][22:23])

{'input_ids': [[2, 537, 13195, 17645, 844, 25194, 2051, 5350, 17, 8, 833, 602, 9243, 602, 1136, 17, 550, 4014, 555, 537, 675, 18312, 1236, 16931, 1622, 2365, 11, 86, 3069, 600, 3873, 1589, 8926, 647, 746, 3938, 2881, 13161, 5441, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], 'labels': [[-100, 0, 1, 0, 0, 0, 2, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, -100, 0, 5, -100, 0, 1, -100, -100, 0, 0, 0, 0, 0, 2, -100, -100, 0, 0, 2, -100, 6, 0, -100]]}

In [18]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1867 [00:00<?, ? examples/s]

Map:   0%|          | 0/467 [00:00<?, ? examples/s]

In [19]:
label_list

['O',
 'B-Object',
 'B-Aspect',
 'I-Object',
 'I-Predicate',
 'B-Predicate',
 'I-Aspect']

In [20]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list)+1)
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of the model checkpoint at cointegrated/rubert-tiny2 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized 

In [21]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [22]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [23]:
metric = load_metric("seqeval")

/home/user/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """Entry point for launching an IPython kernel.


In [24]:
example = ner_data['train'][4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'Aspect': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'Object': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'Predicate': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [25]:
import numpy as np

def compute_metrics(p):
    predictions, labels, inputs = p.predictions, p.label_ids, p.inputs
    predictions = np.argmax(p.predictions, axis=2)

    # send only the first token of each word to the evaluation
    true_predictions = []
    true_labels = []
    for prediction, label, tokens in zip(predictions, labels, inputs):
        true_predictions.append([])
        true_labels.append([])
        for (p, l, t) in zip(prediction, label, tokens):
            if l != -100 and not tokenizer.convert_ids_to_tokens(int(t)).startswith('##'):
                true_predictions[-1].append(label_list[p])
                true_labels[-1].append(label_list[l])

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [26]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [27]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: __index_level_0__, tags, tokens. If __index_level_0__, tags, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 467
  Batch size = 16
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


IndexError: list index out of range

In [ ]:
# разморозка
for param in model.parameters():
    param.requires_grad = True

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
    include_inputs_for_metrics=True,
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()